In [ ]:
import os
from experiment_setup import setups

In [ ]:
# download the dataset, the links can be fund in the README
dataset_path = "./datasets/coveo_ecommerce"
model_path = "./trained_models"

## Run the preprocess script, specific to the dataset you chose

- The preprocessing script in general, executes the following steps:
    - Loads the raw data, with correct types
    - Creates the sessions
    - Removes duplicated items. An item is considered as a duplicate if the preceding (based on time) event in the same session contains the exact same item.
    - Performes iterative support filtering
        - Removes sessions with only one event
        - Removes items with less than 5 events
        - Until the size of the dataset changes


In [ ]:
%run coveo_preproc.py --path $dataset_path

## Use a specific setup for your dataset

In [ ]:
params = setups["coveo"]["params_bprmax"]

In [ ]:
train_path = os.path.join(dataset_path,"coveo_processed_view_train_full.tsv")
test_path = os.path.join(dataset_path,"coveo_processed_view_test.tsv")

In [ ]:
def create_recpack_script(model_name, train_path, test_path, loss, optim, layers, embedding, hidden_size, batch_size, dropout_p_embed, dropout_p_hidden, learning_rate, bpreg, n_epochs, n_sample, m, use_correct_weight_init):
    s_train_test_full = f"CUDA_VISIBLE_DEVICES=0 python ./recpack/main.py --train_path {train_path} --test_path {test_path} --m {m} --n_epochs {n_epochs} --loss {loss} --optim {optim} --layers {layers} --embedding {embedding} --hidden_size {hidden_size} --batch_size {batch_size} --dropout_p_embed {dropout_p_embed} --dropout_p_hidden {dropout_p_hidden} --learning_rate {learning_rate}{f' --n_sample {n_sample}' if loss=='bpr-max' else ''}{' --use_correct_weight_init' if use_correct_weight_init else ''}{f' --bpreg {bpreg}' if loss=='bpr-max' else ''}"
    return s_train_test_full

In [ ]:
loss = params["loss"]
optim = params["optim"]
const_emb = params["constrained_embedding"]
embed = params["embedding"]
final_act = params["final_act"]
layers = params["layers"]
batch_size = params["batch_size"]
dropout_p_embed = params["dropout_p_embed"]
dropout_p_hidden = params["dropout_p_hidden"]
learning_rate = params["learning_rate"]
momentum = params["momentum"]
sample_alpha = params["sample_alpha"]
bpreg = params["bpreg"]
logq = params["logq"]
hidden_act = params["hidden_act"]
n_sample = params["n_sample"]
n_epochs = 5
m = '1 5 10 20'

## Train & test the out-of-the-box model

In [ ]:
train_test_script_oob = create_recpack_script(model_name='recpack_oob_bprmax', train_path=train_path, test_path=test_path,loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_embed, learning_rate=learning_rate, bpreg=1.0, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=False)

In [ ]:
os.system(train_test_script_oob)

## Train & test the minor fix model

In [ ]:
train_test_script_minorfix = create_recpack_script(model_name='recpack_minorfix_bprmax', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_embed, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [ ]:
os.system(train_test_script_minorfix)

## Train & test the major fix model

In [ ]:
train_test_script_majorfix = create_recpack_script(model_name='recpack_majorfix_bprmax', train_path=train_path, test_path=test_path, loss=loss, optim=optim, layers=1, embedding=layers, hidden_size=layers, batch_size=batch_size, dropout_p_embed=dropout_p_embed, dropout_p_hidden=dropout_p_hidden, learning_rate=learning_rate, bpreg=bpreg, n_epochs=n_epochs, n_sample=n_sample, m=m, use_correct_weight_init=True)

In [ ]:
os.system(train_test_script_majorfix)